In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
os.chdir('../..')
sys.path.insert(1, os.path.join(sys.path[0], '../..'))

In [ ]:
MODELS = [
    # 'models/export/models/ssl/voxceleb2/simclr/simclr_proj-none_t-0.03/',
    # 'models/export/models/ssl/voxceleb2/simclr/simclr_proj-none_t-0.03_sup2/',
    # 'models/export/models/ssl/voxceleb2/moco/moco_proj-none_Q-32768_t-0.03_m-0.999/',
    # 'models/export/models/ssl/voxceleb2/moco/moco_proj-none_Q-32768_t-0.03_m-0.999_sup2/',
    # 'models/export/models/ssl/voxceleb2/swav/swav_proj-2048-BN-R-2048-BN-R-512_K-6000_t-0.1/',
    # 'models/export/models/ssl/voxceleb2/swav/swav_proj-2048-BN-R-2048-BN-R-512_K-6000_t-0.1_sup2/',
    # 'models/export/models/ssl/voxceleb2/vicreg/vicreg_proj-2048-BN-R-2048-BN-R-512_inv-1.0_var-1.0_cov-0.1/',
    'models/export/models/ssl/voxceleb2/vicreg/vicreg_proj-2048-BN-R-2048-BN-R-512_inv-1.0_var-1.0_cov-0.1_sup2/',
    'models/export/models/ssl/voxceleb2/dino/dino_plus_nohann/',
    'models/export/models/ssl/voxceleb2/dino/dino_plus_nohann_sup2/',
]

In [ ]:
import torch
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score
import json
from tqdm import tqdm


def compute_nmi(model, K=1251):
    embeddings_path = f'{model}/embeddings_vox1_avg.pt'

    embeddings = torch.load(embeddings_path)

    if len(embeddings) != 153516:
        print(model, len(embeddings))

    X = np.concatenate(list(embeddings.values()))
    y_speaker = [y.split('/')[-3] for y in embeddings.keys()]
    y_video = [y.split('/')[-2] for y in embeddings.keys()]

    kmeans = KMeans(n_clusters=K, init='random', algorithm='lloyd', random_state=0).fit(X)

    nmi_speaker = normalized_mutual_info_score(y_speaker, kmeans.labels_)
    nmi_video = normalized_mutual_info_score(y_video, kmeans.labels_)

    return nmi_speaker, nmi_video

In [ ]:
for model in tqdm(MODELS):
    nmi_speaker, nmi_video = compute_nmi(model)
    print(f'Model: {model} - NMI Speaker: {nmi_speaker} - NMI Video: {nmi_video} - Ratio: {nmi_speaker / nmi_video}')
    with open(f'{model}/nmi_final.json', 'w') as f:
        json.dump({
            "vox1_nmi_speaker": nmi_speaker,
            "vox1_nmi_video": nmi_video
        }, f, indent=4)

In [ ]:
for model in tqdm(MODELS):
    nmi_speaker, nmi_video = compute_nmi(model)
    print(f'Model: {model} - NMI Speaker: {nmi_speaker} - NMI Video: {nmi_video} - Ratio: {nmi_speaker / nmi_video}')
    with open(f'{model}/nmi_final.json', 'w') as f:
        json.dump({
            "vox1_nmi_speaker": nmi_speaker,
            "vox1_nmi_video": nmi_video
        }, f, indent=4)

In [ ]:
for model in tqdm(MODELS):
    print(model)
    nmi_speaker, nmi_video = compute_nmi(model)
    print(nmi_speaker)
    print(nmi_video)
    print(nmi_speaker / nmi_video)

    with open(f'{model}/nmi_new_1251.json', 'w') as f:
        json.dump({
            "vox1_nmi_speaker": nmi_speaker,
            "vox1_nmi_video": nmi_video
        }, f, indent=4)

In [ ]:
simclr
0.8541255921948855
0.9127909957968735
0.9357296425226317

simclr sup
0.8811415956798431
0.8833679884404396 
0.9974796542440629

In [ ]:
for model in tqdm(MODELS):
    print(model)
    nmi_speaker, nmi_video = compute_nmi(model)
    print(nmi_speaker)
    print(nmi_video)
    print(nmi_speaker / nmi_video)

    with open(f'{model}/nmi_new_1251.json', 'w') as f:
        json.dump({
            "vox1_nmi_speaker": nmi_speaker,
            "vox1_nmi_video": nmi_video
        }, f, indent=4)

In [ ]:
 0%|          | 0/4 [00:00<?, ?it/s]
models/export/models/ssl/voxceleb2/moco/moco_proj-none_Q-32768_t-0.03_m-0.999_sup2/
 25%|██▌       | 1/4 [04:14<12:42, 254.00s/it]
0.9119560384840641
0.8273984154045617
1.1021969845544815
models/export/models/ssl/voxceleb2/swav/swav_proj-2048-BN-R-2048-BN-R-512_K-6000_t-0.1_sup2/
 50%|█████     | 2/4 [10:26<10:47, 323.56s/it]
0.8053622362422076
0.7959531469662783
1.01182115971498
models/export/models/ssl/voxceleb2/vicreg/vicreg_proj-2048-BN-R-2048-BN-R-512_inv-1.0_var-1.0_cov-0.1_sup2/
 75%|███████▌  | 3/4 [15:30<05:14, 314.61s/it]
0.8574436747946398
0.8137063916686252
1.053750693829902
models/export/models/ssl/voxceleb2/dino/dino_plus_nohann_proj-none_sup2/
100%|██████████| 4/4 [22:43<00:00, 340.82s/it]
0.6582320584766147
0.7165089238649192
0.9186655414227735